# Setup

In [1]:
MONGODB_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

MONGODB_REPLICA_SET = "replica_set_0"
MONGODB_TOTAL_NODES = 3

MONGODB_NODE_IPS = ["10.15.20.29"] * MONGODB_TOTAL_NODES
MONGODB_NODE_NAMES = [f"mongodb-node-{i + 1}" for i in range(MONGODB_TOTAL_NODES)]
MONGODB_NODE_HOSTNAMES = [
    f"{MONGODB_NODE_NAMES[i]}.gnavar16.vpn.itam.mx" for i in range(MONGODB_TOTAL_NODES)
]
MONGODB_NODE_PORTS = [27010 + (i + 1) for i in range(0, MONGODB_TOTAL_NODES)]

MONGODB_WORKDIR = "/data/db"

MONGO_INITDB_ROOT_USERNAME = "admin"
MONGO_INITDB_ROOT_PASSWORD = "admin"
MONGO_INITDB_DATABASE = "admin"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
MONGODB_LOCAL_CLUSTER_KEY_PATH = os.path.join(DOCKER_MOUNTDIR, "mongo-keyfile")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Stop mongodb-cluster.docker-compose.yml

In [3]:
!docker compose -f mongodb-cluster.docker-compose.yml down -v

open c:\Users\gabon\Desktop\Gabo_ITAM\NO_RELACIONALES\labs\mongodb\mongodb-cluster.docker-compose.yml: The system cannot find the file specified.


In [4]:
import shutil
import stat

if MONGODB_START_FROM_SCRATCH:
    if os.path.exists(MONGODB_LOCAL_CLUSTER_KEY_PATH):
        os.chmod(MONGODB_LOCAL_CLUSTER_KEY_PATH, stat.S_IWRITE)
    shutil.rmtree(DOCKER_MOUNTDIR)
    Path(DOCKER_MOUNTDIR).mkdir(parents=True, exist_ok=True)

# Start mongodb-cluster.docker-compose.yml

In [5]:
import os
import stat
import yaml
import base64
import secrets
from IPython.display import Markdown, display

if not os.path.exists(MONGODB_LOCAL_CLUSTER_KEY_PATH):
    with open(MONGODB_LOCAL_CLUSTER_KEY_PATH, "w") as f:
        raw_data = secrets.token_bytes(756)
        f.write(base64.b64encode(raw_data).decode("utf-8"))

mongodb_compose_dict = {
    "name": "mongodb-cluster",
    "services": {},
    "networks": {"mongo-cluster": {"driver": "bridge"}},
}

for i in range(MONGODB_TOTAL_NODES):
    mongodb_compose_dict["services"][MONGODB_NODE_NAMES[i]] = {
        "image": "mongo:7.0",
        "container_name": MONGODB_NODE_NAMES[i],
        # "hostname": MONGODB_NODE_HOSTNAMES[i],
        "command": [
            "bash",
            "-c",
            " && ".join(
                [
                    "chown 999:999 /data/configdb/keyfile",
                    "chmod 400 /data/configdb/keyfile",
                    " ".join(
                        [
                            "exec",
                            "docker-entrypoint.sh",
                            "mongod",
                            "--replSet",
                            MONGODB_REPLICA_SET,
                            "--keyFile",
                            "/data/configdb/keyfile",
                            "--bind_ip_all",
                            "--port",
                            f"{MONGODB_NODE_PORTS[i]}",
                        ]
                    ),
                ]
            ),
        ],
        "environment": [
            f"MONGO_INITDB_ROOT_USERNAME={MONGO_INITDB_ROOT_USERNAME}",
            f"MONGO_INITDB_ROOT_PASSWORD={MONGO_INITDB_ROOT_PASSWORD}",
            f"MONGO_INITDB_DATABASE={MONGO_INITDB_DATABASE}",
        ],
        "volumes": [
            f"{os.path.join(DOCKER_MOUNTDIR, MONGODB_NODE_NAMES[i])}:/data/db",
            f"{os.path.join(DOCKER_MOUNTDIR, 'mongo-keyfile')}:/data/configdb/keyfile",
        ],
        "networks": ["mongo-cluster"],
        "ports": [f"{MONGODB_NODE_PORTS[i]}:{MONGODB_NODE_PORTS[i]}"],
        "extra_hosts": [f"{DOCKER_INTERNAL_HOST}:host-gateway"],
        "dns": DOCKER_DNS,
        "deploy": {"resources": {"limits": {"cpus": "1.0", "memory": "1024M"}}},
        "healthcheck": {
            "test": [
                "CMD",
                "mongosh",
                "--port",
                f"{MONGODB_NODE_PORTS[i]}",
                "--quiet",
                "--eval",
                "db.adminCommand('ping')",
            ],
            "interval": "10s",
            "timeout": "10s",
            "retries": 30,
            "start_period": "30s",
        },
        "depends_on": {
            MONGODB_NODE_NAMES[j]: {"condition": "service_started"} for j in range(0, i)
        },
    }

mongodb_compose_yaml_path = os.path.join(
    LOCALHOST_WORKDIR, "mongodb-cluster.docker-compose.yml"
)
mongodb_compose_yaml_contents = yaml.dump(
    mongodb_compose_dict, default_flow_style=False, sort_keys=False, indent=4
)
with open(mongodb_compose_yaml_path, "w") as f:
    f.write(mongodb_compose_yaml_contents)

(print(f"Successfully created: '{os.path.relpath(mongodb_compose_yaml_path)}'"),)
display(Markdown(f"```yaml\n{mongodb_compose_yaml_contents}\n```"))

Successfully created: 'mongodb-cluster.docker-compose.yml'


```yaml
name: mongodb-cluster
services:
    mongodb-node-1:
        image: mongo:7.0
        container_name: mongodb-node-1
        command:
        - bash
        - -c
        - chown 999:999 /data/configdb/keyfile && chmod 400 /data/configdb/keyfile
            && exec docker-entrypoint.sh mongod --replSet replica_set_0 --keyFile
            /data/configdb/keyfile --bind_ip_all --port 27011
        environment:
        - MONGO_INITDB_ROOT_USERNAME=admin
        - MONGO_INITDB_ROOT_PASSWORD=admin
        - MONGO_INITDB_DATABASE=admin
        volumes:
        - .\mount\mongodb-node-1:/data/db
        - .\mount\mongo-keyfile:/data/configdb/keyfile
        networks:
        - mongo-cluster
        ports:
        - 27011:27011
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: &id001
        - 10.15.20.1
        deploy:
            resources:
                limits:
                    cpus: '1.0'
                    memory: 1024M
        healthcheck:
            test:
            - CMD
            - mongosh
            - --port
            - '27011'
            - --quiet
            - --eval
            - db.adminCommand('ping')
            interval: 10s
            timeout: 10s
            retries: 30
            start_period: 30s
        depends_on: {}
    mongodb-node-2:
        image: mongo:7.0
        container_name: mongodb-node-2
        command:
        - bash
        - -c
        - chown 999:999 /data/configdb/keyfile && chmod 400 /data/configdb/keyfile
            && exec docker-entrypoint.sh mongod --replSet replica_set_0 --keyFile
            /data/configdb/keyfile --bind_ip_all --port 27012
        environment:
        - MONGO_INITDB_ROOT_USERNAME=admin
        - MONGO_INITDB_ROOT_PASSWORD=admin
        - MONGO_INITDB_DATABASE=admin
        volumes:
        - .\mount\mongodb-node-2:/data/db
        - .\mount\mongo-keyfile:/data/configdb/keyfile
        networks:
        - mongo-cluster
        ports:
        - 27012:27012
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '1.0'
                    memory: 1024M
        healthcheck:
            test:
            - CMD
            - mongosh
            - --port
            - '27012'
            - --quiet
            - --eval
            - db.adminCommand('ping')
            interval: 10s
            timeout: 10s
            retries: 30
            start_period: 30s
        depends_on:
            mongodb-node-1:
                condition: service_started
    mongodb-node-3:
        image: mongo:7.0
        container_name: mongodb-node-3
        command:
        - bash
        - -c
        - chown 999:999 /data/configdb/keyfile && chmod 400 /data/configdb/keyfile
            && exec docker-entrypoint.sh mongod --replSet replica_set_0 --keyFile
            /data/configdb/keyfile --bind_ip_all --port 27013
        environment:
        - MONGO_INITDB_ROOT_USERNAME=admin
        - MONGO_INITDB_ROOT_PASSWORD=admin
        - MONGO_INITDB_DATABASE=admin
        volumes:
        - .\mount\mongodb-node-3:/data/db
        - .\mount\mongo-keyfile:/data/configdb/keyfile
        networks:
        - mongo-cluster
        ports:
        - 27013:27013
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '1.0'
                    memory: 1024M
        healthcheck:
            test:
            - CMD
            - mongosh
            - --port
            - '27013'
            - --quiet
            - --eval
            - db.adminCommand('ping')
            interval: 10s
            timeout: 10s
            retries: 30
            start_period: 30s
        depends_on:
            mongodb-node-1:
                condition: service_started
            mongodb-node-2:
                condition: service_started
networks:
    mongo-cluster:
        driver: bridge

```

In [6]:
!docker compose -f mongodb-cluster.docker-compose.yml up -d --wait

 mongodb-node-3 Pulling 
 mongodb-node-2 Pulling 
 mongodb-node-1 Pulling 
 47c219b2d4f5 Pulling fs layer 
 a4867e714827 Pulling fs layer 
 7e49dc6156b0 Pulling fs layer 
 d0fc8d2469ec Pulling fs layer 
 d7381d7512ed Pulling fs layer 
 a4c2e1a296a6 Pulling fs layer 
 929ce23bedbe Pulling fs layer 
 487128c0e848 Pulling fs layer 
 7e49dc6156b0 Pulling fs layer 
 d0fc8d2469ec Pulling fs layer 
 487128c0e848 Pulling fs layer 
 a4c2e1a296a6 Pulling fs layer 
 47c219b2d4f5 Pulling fs layer 
 a4867e714827 Pulling fs layer 
 d7381d7512ed Pulling fs layer 
 929ce23bedbe Pulling fs layer 
 d0fc8d2469ec Pulling fs layer 
 47c219b2d4f5 Pulling fs layer 
 a4867e714827 Pulling fs layer 
 7e49dc6156b0 Pulling fs layer 
 d7381d7512ed Pulling fs layer 
 929ce23bedbe Pulling fs layer 
 487128c0e848 Pulling fs layer 
 a4c2e1a296a6 Pulling fs layer 
 47c219b2d4f5 Downloading [==================================================>]     116B/116B
 47c219b2d4f5 Downloading [====================================

In [7]:
import time
from pymongo import MongoClient
from pymongo.errors import OperationFailure

MONGODB_PRIMARY_SELECTION_TIMEOUT_SECONDS = 30
client_options = {"directConnection": True, "serverSelectionTimeoutMS": 5000}

init_client = MongoClient(
    f"mongodb://{MONGO_INITDB_ROOT_USERNAME}:{MONGO_INITDB_ROOT_PASSWORD}@{MONGODB_NODE_HOSTNAMES[0]}:{MONGODB_NODE_PORTS[0]}/",
    **client_options,
)
try:
    print(f"🚀 Initializing Replica Set: '{MONGODB_REPLICA_SET}'")
    init_client.admin.command(
        "replSetInitiate",
        {
            "_id": MONGODB_REPLICA_SET,
            "members": [
                {
                    "_id": i,
                    "host": f"{MONGODB_NODE_HOSTNAMES[i]}:{MONGODB_NODE_PORTS[i]}",
                }
                for i in range(MONGODB_TOTAL_NODES)
            ],
        },
    )
    print("✅ Initiation command accepted.")
except OperationFailure as e:
    if "already initialized" in str(e).lower():
        print("⚠️ Cluster is already initiated. Verifying health...")
    else:
        raise

start_time = time.time()
primary_found = False
print(f"⏳ Waiting for Primary (Timeout: {MONGODB_PRIMARY_SELECTION_TIMEOUT_SECONDS}s)")
while time.time() - start_time < MONGODB_PRIMARY_SELECTION_TIMEOUT_SECONDS:
    for i in range(MONGODB_TOTAL_NODES):
        try:
            with MongoClient(
                f"mongodb://{MONGODB_NODE_HOSTNAMES[i]}:{MONGODB_NODE_PORTS[i]}/",
                **client_options,
            ) as node_check:
                res = node_check.admin.command("hello")
                if res.get("isWritablePrimary") or res.get("ismaster"):
                    primary_found = True
                    elapsed = round(time.time() - start_time, 2)
                    print(
                        f"\n🌟 Primary Elected: {res.get('me')} (Found at {MONGODB_NODE_HOSTNAMES[i]}:{MONGODB_NODE_PORTS[i]} in {elapsed}s)"
                    )
                    break
        except Exception:
            continue
    if primary_found:
        break
    print(f"Still electing... [{int(time.time() - start_time)}s]", end="\r")
    time.sleep(0.05)

if not primary_found:
    status = init_client.admin.command("replSetGetStatus")
    print("\n❌ Timeout reached. Current Node States:")
    for m in status.get("members", []):
        print(f" - {m['name']}: {m['stateStr']}")
    raise TimeoutError("Replica Set failed to elect a Primary.")
else:
    status = init_client.admin.command("replSetGetStatus")
    print(f"\nCluster '{MONGODB_REPLICA_SET}' Status Summary:")
    for m in status["members"]:
        icon = "🟢" if m["health"] == 1 else "🔴"
        print(f"{icon} {m['name']:<35} | {m['stateStr']:<10}")

🚀 Initializing Replica Set: 'replica_set_0'
✅ Initiation command accepted.
⏳ Waiting for Primary (Timeout: 30s)
Still electing... [10s]
🌟 Primary Elected: mongodb-node-3.gnavar16.vpn.itam.mx:27013 (Found at mongodb-node-3.gnavar16.vpn.itam.mx:27013 in 11.05s)

Cluster 'replica_set_0' Status Summary:
🟢 mongodb-node-1.gnavar16.vpn.itam.mx:27011 | SECONDARY 
🟢 mongodb-node-2.gnavar16.vpn.itam.mx:27012 | SECONDARY 
🟢 mongodb-node-3.gnavar16.vpn.itam.mx:27013 | PRIMARY   


In [ ]:
# docker exec -it mongodb-node-1 mongosh --port 27011 -u "admin" -p "admin" --authenticationDatabase "admin"
# docker exec -it mongodb-node-2 mongosh --port 27012 -u "admin" -p "admin" --authenticationDatabase "admin"
# docker exec -it mongodb-node-3 mongosh --port 27013 -u "admin" -p "admin" --authenticationDatabase "admin"

# !docker exec -i mongodb-node-1 mongosh --port 27011 -u "admin" -p "admin" --authenticationDatabase "admin" --quiet --eval "db.estudiantes.find({ promedio: { $gt: 9.5 } }).explain('allPlansExecution')"
# !docker exec -i mongodb-node-2 mongosh --port 27012 -u "admin" -p "admin" --authenticationDatabase "admin" --quiet --eval "db.estudiantes.find({ promedio: { $gt: 9.5 } }).explain('allPlansExecution')"
# !docker exec -i mongodb-node-3 mongosh --port 27013 -u "admin" -p "admin" --authenticationDatabase "admin" --quiet --eval "db.estudiantes.find({ promedio: { $gt: 9.5 } }).explain('allPlansExecution')"